# Prendre en main LangChain pas à pas

Objectif de ce notebook :  
- Comprendre les **briques de base** de LangChain (LLM, prompt, chain).  
- Visualiser un **schéma simple** de pipeline.  
- Construire un mini **pipeline de question/réponse**, puis un mini **pipeline RAG simplifié**.

> Remarque : ce notebook suppose que vous avez une clé API (`OPENAI_API_KEY`) définie dans vos variables d’environnement.

## Les briques de base de LangChain

On va manipuler 3 briques principales :

1. **LLM** : le modèle de langage (par ex. `gpt-4o-mini`)  
2. **Prompt** : la manière dont on parle au modèle (system + user + variables)  
3. **Chain** : l’assemblage **déclaratif** des étapes (pipeline de traitement)

On peut imaginer ce schéma très simple :

```text
              +---------------------------+
User input -> |   PromptTemplate          | -> texte formaté
              +---------------------------+
                           |
                           v
                     +-----------+
                     |   LLM     | -> réponse brute du modèle
                     +-----------+
                           |
                           v
                   +------------------+
                   | Output Parser    | -> string / JSON / etc.
                   +------------------+
```  
LangChain permet de **décrire ce pipeline en Python**, plutôt que de tout reprogrammer à la main.

## 🔧 Installation (à faire une fois dans votre environnement)

Décommentez et exécutez la cellule ci-dessous si besoin.

In [ ]:
# !pip install -U langchain-core langchain-openai
# Optionnel : outils supplémentaires
# !pip install -U langchain-text-splitters

## ⚙️ Initialisation du LLM

On crée un objet `ChatOpenAI` qui est la brique "modèle de langage" côté LangChain.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from getpass import getpass

# IMPORTANT: Configurez votre clé API OpenAI dans vos variables d'environnement
# Vous pouvez les définir via un fichier .env ou en les exportant dans votre shell:
# export OPENAI_API_KEY="your-key-here"
# Ou utilisez getpass pour la saisir de manière sécurisée:
# os.environ["OPENAI_API_KEY"] = getpass("Entrez votre clé API OpenAI: ")

# Vérification que la clé est définie
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("OPENAI_API_KEY doit être définie dans vos variables d'environnement. "
                     "Voir le README.md pour les instructions de configuration.")

model = ChatOpenAI(
    model="gpt-4o-mini",  
    temperature=0.2,
)
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10c58b9a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10c58bf10>, root_client=<openai.OpenAI object at 0x10c589600>, root_async_client=<openai.AsyncOpenAI object at 0x10c58ab00>, model_name='gpt-4o-mini', temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'))

## 1️⃣ Premier appel au LLM via LangChain

On commence par l’usage le plus simple : on envoie une liste de messages au modèle.

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Tu es un assistant pédagogique, clair et concis."),
    HumanMessage(content="Explique en deux phrases ce qu'est un modèle de langage."),
]

response = model.invoke(messages)
print(response.content)

Un modèle de langage est un système informatique conçu pour comprendre et générer du texte en se basant sur des données linguistiques. Il utilise des algorithmes d'apprentissage automatique pour prédire la probabilité des mots ou des phrases dans un contexte donné.


## 2️⃣ Séparer le **prompt** et le **modèle**

Plutôt que de construire les messages à la main, on décrit un **template de conversation**.

On utilise `ChatPromptTemplate` pour définir :
- le rôle système (persona),
- le message utilisateur avec une **variable `{question}`**.

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"Tu es un assistant IA qui vulgarise pour un public de développeurs curieux."),
        ("human", "Question : {question}"),
    ]
)


# LCEL: LangChain Expression Language
chain = prompt | model 

answer = chain.invoke({"question": "Quelle est la différence entre un LLM et LangChain ?"})
print(answer)



content="Un LLM (Large Language Model) et LangChain sont deux concepts différents, mais ils peuvent travailler ensemble dans le domaine du traitement du langage naturel.\n\n### LLM (Large Language Model)\nUn LLM est un modèle d'intelligence artificielle entraîné sur de vastes ensembles de données textuelles. Ces modèles, comme GPT-3 ou GPT-4, sont capables de comprendre et de générer du texte de manière cohérente. Ils peuvent être utilisés pour diverses tâches, telles que :\n\n- La génération de texte\n- La traduction\n- La réponse à des questions\n- La rédaction de contenu\n\nLes LLMs fonctionnent en prédisant le mot suivant dans une phrase, en se basant sur le contexte fourni par les mots précédents.\n\n### LangChain\nLangChain, en revanche, est une bibliothèque conçue pour faciliter l'intégration et l'utilisation des LLMs dans des applications plus complexes. Elle permet de créer des chaînes de traitement de texte qui combinent plusieurs étapes, comme :\n\n- L'interaction avec des L

In [22]:
parser = StrOutputParser()

parser.invoke(answer)

"Un LLM (Large Language Model) et LangChain sont deux concepts différents, mais ils peuvent travailler ensemble dans le domaine du traitement du langage naturel.\n\n### LLM (Large Language Model)\nUn LLM est un modèle d'intelligence artificielle entraîné sur de vastes ensembles de données textuelles. Ces modèles, comme GPT-3 ou GPT-4, sont capables de comprendre et de générer du texte de manière cohérente. Ils peuvent être utilisés pour diverses tâches, telles que :\n\n- La génération de texte\n- La traduction\n- La réponse à des questions\n- La rédaction de contenu\n\nLes LLMs fonctionnent en prédisant le mot suivant dans une phrase, en se basant sur le contexte fourni par les mots précédents.\n\n### LangChain\nLangChain, en revanche, est une bibliothèque conçue pour faciliter l'intégration et l'utilisation des LLMs dans des applications plus complexes. Elle permet de créer des chaînes de traitement de texte qui combinent plusieurs étapes, comme :\n\n- L'interaction avec des LLMs\n- L

👉 Ici, on vient d’instancier le **schéma précédent** :

```text
variables -> PromptTemplate -> LLM -> OutputParser -> string
```

LangChain gère pour nous :
- la création des messages,
- l’appel modèle,
- la conversion de la réponse en `str`.

## 3️⃣ Ajouter un peu de "contexte" (mini-RAG sans base vectorielle)

On simule ici un RAG **très simplifié** :  
- un mini "corpus" en mémoire,  
- une fonction de récupération naïve (recherche par mots-clés),  
- un prompt qui combine **question + contexte**.

L’objectif est juste de montrer la **brique "retrieval"** dans le pipeline.

In [23]:
# Mini "base de connaissances" en mémoire
DOCUMENTS = [
    {
        "title": "Agents déterministes",
        "content": "Un agent déterministe suit un scénario fixe, sans appel à un modèle de langage à chaque étape."
    },
    {
        "title": "Agents IA (LLM)",
        "content": "Un agent IA utilise un modèle de langage pour décider dynamiquement des actions à chaque étape."
    },
    {
        "title": "LangChain",
        "content": "LangChain est un framework qui aide à structurer les appels aux LLM et à intégrer des données, des outils et de la mémoire."
    },
]

def simple_retriever(query: str, k: int = 2) -> str:
    """
    Très simple :
    - calcule un score basé sur le nombre de tokens communs
    - renvoie les k meilleurs documents concaténés
    """
    tokens = set(query.lower().split())
    scored = []
    for doc in DOCUMENTS:
        score = len(tokens.intersection(set(doc["content"].lower().split())))
        scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    top_docs = [doc["content"] for score, doc in scored[:k] if score > 0]
    if not top_docs:
        return "Aucun document pertinent trouvé dans la base locale."
    return "\n\n---\n\n".join(top_docs)

print(simple_retriever("Quelle différence entre agent IA et agent déterministe ?"))

Un agent déterministe suit un scénario fixe, sans appel à un modèle de langage à chaque étape.

---

Un agent IA utilise un modèle de langage pour décider dynamiquement des actions à chaque étape.


### 🔗 Chaîne RAG simplifiée 

On utilise `RunnableLambda` et `RunnablePassthrough` pour construire un pipeline :

```text
question
  ├──> (pass-through) ────────────────┐
  └──> retriever(question) -> contexte┤
                                      v
                           Prompt(question + contexte)
                                    |
                                    v
                                 LLM
                                    |
                                    v
                              OutputParser
```

In [24]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Étape 1 : préparer le "retriever" comme Runnable
retriever_runnable = RunnableLambda(lambda q: simple_retriever(q))

# Étape 2 : définir le prompt qui intègre question + contexte
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un assistant qui répond en t'appuyant STRICTEMENT sur le contexte fourni.\n"
            "Si une information n'est pas dans le contexte, dis-le explicitement."
        ),
        (
            "human",
            "Contexte :\n{context}\n\n"
            "Question utilisateur : {question}"
        ),
    ]
)

rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever_runnable,
    }
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "Explique la différence entre un agent déterministe et un agent IA."
print(rag_chain.invoke(question))

Un agent déterministe suit un scénario fixe et préétabli, sans faire appel à un modèle de langage à chaque étape. En revanche, un agent IA, comme ceux qui utilisent des modèles de langage, peut adapter ses réponses et ses actions en fonction des données et des interactions en temps réel.


## ✅ Récap
 
Briques présentées dans ce notebook :
  - modèle de langage (`ChatOpenAI`),
  - prompts (`ChatPromptTemplate`),
  - pipeline (`chain = prompt | model | parser`),
  - retrieval/RAG (même très simplifié).

Pour la suite nous allons aborder :  
- l’orchestration **multi-étapes** avec LangGraph,  
- puis le **pattern d’agent** plus avancé dans le dernier notebook.